In [1]:
import geopandas as gpd
import pandas as pd
import sys
sys.path.append('../')
from data_processing import load_nuts_boundaries, calculate_clc_land_area_by_region, aggregate_clc_land_area_categories, save_data

download clc from: https://land.copernicus.eu/en/products/corine-land-cover/clc2018#download



In [2]:
land_use = gpd.read_file(r"/gb_pv_capacity_model/data/U2018_CLC2018_V2020_20u1.gdb")
land_use

,Code_18,Remark,Area_Ha,ID,Shape_Length,Shape_Area,geometry
0,111,NaN,130.863654,EU_1,10902.412471,1.308637e+06,"MULTIPOLYGON (((1917182.160 943608.860, 191719..."
1,111,NaN,53.744524,EU_2,6329.456389,5.374452e+05,"MULTIPOLYGON (((1953122.840 950507.440, 195304..."
2,111,NaN,30.719104,EU_3,3371.774719,3.071910e+05,"MULTIPOLYGON (((1956709.150 951094.550, 195676..."
3,111,NaN,50.201782,EU_4,4869.883061,5.020178e+05,"MULTIPOLYGON (((1805587.500 950821.540, 180558..."
4,111,NaN,481.848803,EU_5,14658.685372,4.818488e+06,"MULTIPOLYGON (((1792547.840 952643.380, 179256..."
...,...,...,...,...,...,...,...
2375401,512,NaN,807.040751,EU_2375402,36492.882472,8.070408e+06,"MULTIPOLYGON (((4958287.940 5319798.460, 49583..."
2375402,512,NaN,141.362708,EU_2375403,10504.564682,1.413627e+06,"MULTIPOLYGON (((4955932.460 5325540.080, 49559..."
2375403,512,NaN,246.634201,EU_2375404,9835.137493,2.466342e+06,"MULTIPOLYGON (((4956804.160 5330567.090, 49568..."
2375404,512,NaN,41.845799,EU_2375405,3887.825769,4.184580e+05,"MULTIPOLYGON (((4957909.360 5364282.460, 49579..."


In [3]:
nuts = load_nuts_boundaries("/gb_pv_capacity_model/data/nuts.shp.zip", download=True)
nuts

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
0,AL,0,AL,Shqipëria,Shqipëria,0.0,0,0,"MULTIPOLYGON (((19.75628 42.63384, 19.75005 42..."
1,CZ,0,CZ,Česko,Česko,0.0,0,0,"POLYGON ((14.33499 51.04007, 14.34494 51.03908..."
2,DE,0,DE,Deutschland,Deutschland,0.0,0,0,"MULTIPOLYGON (((10.45444 47.55580, 10.43954 47..."
3,DK,0,DK,Danmark,Danmark,0.0,0,0,"MULTIPOLYGON (((15.19309 55.32015, 15.19056 55..."
4,CY,0,CY,Kýpros,Κύπρος,0.0,0,0,"MULTIPOLYGON (((34.60609 35.70767, 34.60060 35..."
...,...,...,...,...,...,...,...,...,...
2005,NO0B1,3,NO,Jan Mayen,Jan Mayen,3.0,3,1,"POLYGON ((-9.08295 70.86469, -9.07843 70.86831..."
2006,EE009,3,EE,Kesk-Eesti,Kesk-Eesti,4.0,3,1,"MULTIPOLYGON (((25.98343 59.62115, 25.99837 59..."
2007,NO0,1,NO,Norge,Norge,0.0,0,0,"MULTIPOLYGON (((33.09131 80.24908, 33.09929 80..."
2008,NO0B,2,NO,Jan Mayen and Svalbard,Jan Mayen and Svalbard,NaN,0,0,"MULTIPOLYGON (((31.83002 80.07989, 31.85352 80..."


In [4]:
# select nuts3 regions
nuts3 = nuts[(nuts["LEVL_CODE"] == 3) & (nuts["CNTR_CODE"] == "UK")].rename(
    columns={'NUTS_ID': 'nuts_cd', 'CNTR_CODE': 'country_cd', 'NAME_LATN': 'nuts_name'}
)
nuts3

,nuts_cd,LEVL_CODE,country_cd,nuts_name,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
1417,UKM92,3,UK,Dumfries & Galloway,Dumfries & Galloway,4.0,3,1,"MULTIPOLYGON (((-3.97411 55.45872, -3.96949 55..."
1574,UKK13,3,UK,Gloucestershire,Gloucestershire,4.0,2,1,"POLYGON ((-1.74688 52.09325, -1.74240 52.09199..."
1575,UKK14,3,UK,Swindon,Swindon,4.0,2,2,"MULTIPOLYGON (((-1.69076 51.66327, -1.68902 51..."
1576,UKL22,3,UK,Cardiff and Vale of Glamorgan,Cardiff and Vale of Glamorgan,4.0,1,1,"MULTIPOLYGON (((-3.15738 51.55724, -3.15257 51..."
1577,UKF14,3,UK,Nottingham,Nottingham,4.0,1,3,"POLYGON ((-1.18275 53.01857, -1.18443 53.01446..."
...,...,...,...,...,...,...,...,...,...
1999,UKJ43,3,UK,Kent Thames Gateway,Kent Thames Gateway,4.0,1,1,"MULTIPOLYGON (((0.72791 51.42288, 0.73150 51.4..."
2000,UKN08,3,UK,"Newry, Mourne and Down","Newry, Mourne and Down",4.0,3,1,"MULTIPOLYGON (((-5.81730 54.49184, -5.80878 54..."
2001,UKM64,3,UK,Na h-Eileanan Siar (Western Isles),Na h-Eileanan Siar (Western Isles),4.0,3,1,"MULTIPOLYGON (((-5.82046 59.12544, -5.81955 59..."
2002,UKG37,3,UK,Sandwell,Sandwell,4.0,1,3,"POLYGON ((-1.98893 52.56297, -1.97521 52.55595..."


In [5]:
# select nuts0 regions
nuts0 = nuts[(nuts["LEVL_CODE"] == 0) & (nuts["CNTR_CODE"] == "UK")].rename(
    columns={'NUTS_ID': 'nuts_cd', 'CNTR_CODE': 'country_cd', 'NAME_LATN': 'nuts_name'}
)
nuts0

,nuts_cd,LEVL_CODE,country_cd,nuts_name,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
60,UK,0,UK,United Kingdom,United Kingdom,0.0,0,0,"MULTIPOLYGON (((-3.41143 58.64020, -3.40836 58..."


In [6]:
processed_clc_by_nuts3= calculate_clc_land_area_by_region(nuts3, land_use, 'nuts_cd', 'Code_18')

In [7]:
processed_clc_by_nuts0= calculate_clc_land_area_by_region(nuts0, land_use, 'nuts_cd', 'Code_18')

In [8]:
# replace nan values with zero
processed_clc_by_nuts3.fillna(0, inplace=True)
processed_clc_by_nuts0.fillna(0, inplace=True)

In [12]:
# drop clc area which is approximatly equal to region area and 999 which is the code for no data
processed_clc_by_nuts3 = processed_clc_by_nuts3.drop(columns=['clc_area'])
processed_clc_by_nuts0 = processed_clc_by_nuts0.drop(columns=['clc_area'])
# rename region area to country area
processed_clc_by_nuts0.rename(columns={'region_area': 'country_area', 'nuts_cd':'country_cd'}, inplace=True)

In [15]:
processed_clc_by_nuts3 = aggregate_clc_land_area_categories(processed_clc_by_nuts3)
processed_clc_by_nuts0 = aggregate_clc_land_area_categories(processed_clc_by_nuts0)

In [17]:
processed_clc_by_nuts3
# processed_clc_by_nuts0

,nuts_cd,111,112,121,122,123,124,131,132,133,...,33,41,42,51,52,1,2,3,4,5
0,UKC11,2.211872e+06,6.177013e+07,2.680655e+07,1.908275e+06,2.628929e+06,1.345714e+06,7.365798e+05,0.000000e+00,0.000000,...,5.300749e+05,2.037981e+06,3.247164e+06,2.284093e+06,5.350373e+05,1.111774e+08,1.623366e+08,1.852333e+07,5.285145e+06,2.819130e+06
1,UKC12,3.474269e+06,5.247787e+07,1.900607e+07,4.135874e+06,4.345735e+06,0.000000e+00,1.597518e+06,0.000000e+00,0.000000,...,0.000000e+00,5.868856e+05,1.496167e+06,7.367216e+05,2.596980e+05,9.866075e+07,1.479055e+08,4.969158e+07,2.083052e+06,9.964196e+05
2,UKC13,1.305409e+06,2.288994e+07,3.805323e+06,0.000000e+00,0.000000e+00,1.067770e+06,3.771955e+04,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.391480e+07,1.621511e+08,1.567312e+06,0.000000e+00,0.000000e+00
3,UKC14,1.413122e+06,1.197741e+08,1.946216e+07,7.813483e+05,2.368713e+05,0.000000e+00,9.064198e+06,0.000000e+00,0.000000,...,4.053958e+07,2.452744e+08,2.467943e+05,8.389187e+06,8.215339e+05,1.777059e+08,1.219292e+09,5.818351e+08,2.455212e+08,9.210721e+06
4,UKC21,1.422638e+06,7.997184e+07,2.031969e+07,0.000000e+00,4.480040e+05,1.305876e+05,6.928270e+06,2.940841e+05,844729.273050,...,9.842561e+06,2.975796e+08,5.757825e+06,2.052757e+07,2.653919e+06,1.605360e+08,2.720423e+09,1.824350e+09,3.033374e+08,2.318149e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,UKN0C,8.372426e+05,3.866786e+07,1.147410e+07,0.000000e+00,0.000000e+00,2.942096e+06,3.271579e+06,2.886583e+05,447049.420250,...,5.587074e+05,1.361388e+08,1.926712e+06,4.998481e+06,5.117300e+06,7.088345e+07,1.208689e+09,5.576938e+08,1.380655e+08,1.011578e+07
175,UKN0D,0.000000e+00,3.515044e+07,1.126067e+07,0.000000e+00,0.000000e+00,5.902807e+06,1.837514e+06,4.401406e+05,796443.273026,...,0.000000e+00,3.134511e+06,4.116472e+04,1.571098e+08,2.455134e+03,6.082647e+07,4.788524e+08,2.772771e+07,3.175676e+06,1.571122e+08
176,UKN0E,5.259916e+05,3.349062e+07,6.876741e+06,0.000000e+00,0.000000e+00,0.000000e+00,2.365835e+06,6.350179e+05,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,6.010878e+06,0.000000e+00,4.886628e+07,4.415147e+08,1.378189e+07,0.000000e+00,6.010878e+06
177,UKN0F,2.323608e+06,3.343679e+07,6.290727e+06,3.970738e+05,1.226938e+06,0.000000e+00,2.681115e+06,1.293208e+06,396807.050600,...,0.000000e+00,3.283461e+07,1.053463e+06,4.557806e+06,1.324340e+06,5.488552e+07,7.133882e+08,2.413377e+08,3.388807e+07,5.882145e+06


In [ ]:
save_data({'clc_by_nuts0.csv': nuts0_capacity, 'clc_by_nuts3.csv':nuts3_capacity})